In [37]:
import pandas as pd
import numpy as np
df = pd.read_excel('Input.xlsx')
df.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Load the input.xlsx file
df = pd.read_excel('Input.xlsx')

# Function to extract article text from a URL
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article = soup.find('article')
    if article:
        title = article.find('h1').text.strip()
        text = ' '.join([p.text.strip() for p in article.find_all('p')])
        return title + '\n\n' + text
    return None

# Create a directory to save the extracted articles
os.makedirs('extracted_articles', exist_ok=True)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    extracted_text = extract_article_text(url)
    if extracted_text:
        with open(f'extracted_articles/{url_id}.txt', 'w', encoding='utf-8') as file:
            file.write(extracted_text)


In [39]:
import os
import pandas as pd

# List all text files in the 'extracted_articles' directory
files = os.listdir('extracted_articles')

# Create a list to store the extracted article text and URL_IDs
extracted_data = []

# Iterate over each text file
for file in files:
    url_id = file.split('.')[0]
    with open(f'extracted_articles/{file}', 'r', encoding='utf-8') as f:
        article_text = f.read()
    extracted_data.append({'URL_ID': url_id, 'Article_Text': article_text})

# Create a DataFrame from the extracted_data list
extracted_df = pd.DataFrame(extracted_data)

# Load the input.xlsx file
df = pd.read_excel('Input.xlsx')

# Merge the extracted_df with df on 'URL_ID'
df = pd.merge(df, extracted_df, on='URL_ID', how='left')

# Save the updated DataFrame back to input.xlsx
df.to_csv('input.csv', index=False)

In [40]:
extracted_df = pd.read_csv('input.csv')
extracted_df.head()

,URL_ID,URL,Article_Text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...


In [41]:
extracted_df.isnull().sum()

URL_ID          0
URL             0
Article_Text    2
dtype: int64

In [42]:
clean_df = extracted_df.dropna()

In [43]:
clean_df.isnull().sum()

URL_ID          0
URL             0
Article_Text    0
dtype: int64

# Remove Stopwords

In [50]:
import nltk
nltk.download('punkt')


# Assuming you have multiple stopword files
stop_words_files = ['StopWords/StopWords_Auditor.txt', 
                    'StopWords/StopWords_Currencies.txt',
                    'StopWords/StopWords_DatesandNumbers.txt',
                    'StopWords/StopWords_Generic.txt', 
                    'StopWords/StopWords_GenericLong.txt',
                    'StopWords/StopWords_Geographic.txt', 
                    'StopWords/StopWords_Names.txt']

for file_path in stop_words_files:
    print(file_path)  # Print the file path
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            stopwords_list = file.read().splitlines()
            custom_stopwords.update(stopwords_list)
    except UnicodeDecodeError:
        print(f"Error reading file {file_path}. Skipping...")


# Assuming you have already loaded the stopwords and have your DataFrame 'df'

# Create a new column for the filtered text
df['Article_Text_Filtered'] = ""

# Remove stopwords from the 'Article_Text' column and store the filtered text in the new column
for index, row in df.iterrows():
    text = row['Article_Text']
    if isinstance(text, str):  # Check if text is a string
        word_tokens = word_tokenize(text.lower())
        filtered_text = ' '.join([word for word in word_tokens if word not in custom_stopwords])
        df.loc[index, 'Article_Text_Filtered'] = filtered_text
    else:
        print(f"Skipping row {index} because Article_Text is not a string.")



[nltk_data] Downloading package punkt to /home/danjin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


StopWords/StopWords_Auditor.txt
StopWords/StopWords_Currencies.txt
Error reading file StopWords/StopWords_Currencies.txt. Skipping...
StopWords/StopWords_DatesandNumbers.txt
StopWords/StopWords_Generic.txt
StopWords/StopWords_GenericLong.txt
StopWords/StopWords_Geographic.txt
StopWords/StopWords_Names.txt
Skipping row 35 because Article_Text is not a string.
Skipping row 48 because Article_Text is not a string.


In [52]:
df.head()

,URL_ID,URL,Article_Text,Article_Text_Filtered
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"rising cities impact economy , environment , i..."
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"rising cities impact economy , environment , i..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","internet demand ’ evolution , communication im..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"rise cybercrime effect upcoming future live , ..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,ott platform impact entertainment industry fut...


In [53]:
df.to_csv('clean.csv')

In [58]:
cleandf = pd.read_csv('clean.csv')
cleandf.sample(10)

,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered
96,96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,impact covid-19 pandemic office space co-worki...
43,43,blackassign0044,https://insights.blackcoffer.com/ai-tool-alexa...,How machine learning used in finance and banki...,machine learning finance banking ? ai tools na...
85,85,blackassign0086,https://insights.blackcoffer.com/how-to-increa...,How to increase social media engagement for ma...,increase social media engagement marketers ? s...
98,98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,How COVID-19 is impacting payment preferences?...,covid-19 impacting payment preferences ? pay c...
92,92,blackassign0093,https://insights.blackcoffer.com/travel-and-to...,Travel and Tourism Outlook\n\nThe UN projects ...,travel tourism outlook projects 20-30 % declin...
42,42,blackassign0043,https://insights.blackcoffer.com/future-of-wor...,Future of Work: How AI Has Entered the Workpla...,future work : ai entered workplace artificial ...
3,3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"rise cybercrime effect upcoming future live , ..."
21,21,blackassign0022,https://insights.blackcoffer.com/how-advertise...,How advertisement/marketing affects business.\...,advertisement/marketing affects business . liv...
40,40,blackassign0041,https://insights.blackcoffer.com/ai-human-robo...,How Robo Human will Impact the Future?\n\nIt’s...,robo human impact future ? ’ year 2060. automa...
83,83,blackassign0084,https://insights.blackcoffer.com/how-voice-sea...,How Voice search makes your business a success...,voice search makes business successful busines...


In [59]:
cleandf.isnull().sum()

Unnamed: 0               0
URL_ID                   0
URL                      0
Article_Text             2
Article_Text_Filtered    2
dtype: int64

In [63]:
cleandf1 = cleandf.dropna()

In [64]:
cleandf1.isnull().sum()

Unnamed: 0               0
URL_ID                   0
URL                      0
Article_Text             0
Article_Text_Filtered    0
dtype: int64

# Postive word and negative words

In [68]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Load positive and negative word lists
with open('/home/danjin/Documents/Blackcoffer /MasterDictionary/positive-words.txt', 'r', encoding='latin1') as file:
    positive_words = set(file.read().splitlines())
with open('/home/danjin/Documents/Blackcoffer /MasterDictionary/negative-words.txt', 'r', encoding='latin1') as file:
    negative_words = set(file.read().splitlines())


# Tokenize words in Article_Text_Filtered column
cleandf1['Article_Text_Filtered_Tokenized'] = cleandf1['Article_Text_Filtered'].apply(word_tokenize)

# Function to check if word is positive or negative
def get_positive_negative_words(tokens):
    positive = [word for word in tokens if word in positive_words]
    negative = [word for word in tokens if word in negative_words]
    return positive, negative

# Apply function to tokenize words
cleandf1['Positive_Words'], cleandf1['Negative_Words'] = zip(*cleandf1['Article_Text_Filtered_Tokenized'].apply(get_positive_negative_words))

# Display the dataframe
cleandf1.sample(10)


/tmp/ipykernel_25201/1957713213.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Article_Text_Filtered_Tokenized'] = cleandf1['Article_Text_Filtered'].apply(word_tokenize)
/tmp/ipykernel_25201/1957713213.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Positive_Words'], cleandf1['Negative_Words'] = zip(*cleandf1['Article_Text_Filtered_Tokenized'].apply(get_positive_negative_words))
/tmp/ipykernel_25201/1957713213.py:21: SettingWithCopyWarning: 
A value is trying to be set on a co

,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words
22,22,blackassign0023,https://insights.blackcoffer.com/rising-it-cit...,"Rising IT cities will impact the economy, envi...","rising cities impact economy , environment , i...","[rising, cities, impact, economy, ,, environme...","[positive, eased, smart, miraculously, top, ad...","[negative, worsening, erratic, drought, proble..."
58,58,blackassign0059,https://insights.blackcoffer.com/can-robots-ta...,Can robots tackle late-life loneliness?\n\nIn ...,"robots tackle late-life loneliness ? era , bus...","[robots, tackle, late-life, loneliness, ?, era...","[satisfies, memorable, creative, work, masters...","[loneliness, loneliness, loneliness, lonelines..."
2,2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","internet demand ’ evolution , communication im...","[internet, demand, ’, evolution, ,, communicat...","[astounding, integral, promising, revolutioniz...","[limited, insatiable, lag, latency, lag, obsol..."
84,84,blackassign0085,https://insights.blackcoffer.com/how-the-covid...,How the COVID-19 crisis is redefining jobs and...,covid-19 crisis redefining jobs services ? pan...,"[covid-19, crisis, redefining, jobs, services,...","[long-lasting, work, work, work, work, importa...","[crisis, inevitable, slow, bad, concern, struc..."
40,40,blackassign0041,https://insights.blackcoffer.com/ai-human-robo...,How Robo Human will Impact the Future?\n\nIt’s...,robo human impact future ? ’ year 2060. automa...,"[robo, human, impact, future, ?, ’, year, 2060...","[realistic, good, work, intelligence, safe, si...","[warning, error, hard, funny, joke, joke, joke..."
69,69,blackassign0070,https://insights.blackcoffer.com/how-does-arti...,How does artificial intelligence affect the en...,artificial intelligence affect environment tal...,"[artificial, intelligence, affect, environment...","[intelligence, good, booming, awe, work, varie...","[problems, issues, disaster, drones, problems,..."
59,59,blackassign0060,https://insights.blackcoffer.com/embedding-car...,Embedding care robots into society and practic...,embedding care robots society practice : socio...,"[embedding, care, robots, society, practice, :...","[patient, good, harmony, remarkable, success, ...","[illness, monstrous]"
79,79,blackassign0080,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on sports events a...,impact covid-19 pandemic sports events world ....,"[impact, covid-19, pandemic, sports, events, w...","[significant, super, gold, gold, fans, fans, d...","[disruption, outbreak, tentatively, limited, d..."
67,67,blackassign0068,https://insights.blackcoffer.com/how-you-lead-...,How you lead a project or a team without any t...,lead project team technical expertise ? wonder...,"[lead, project, team, technical, expertise, ?,...","[lead, pep, ardent, top, great, great, great, ...","[lame, naïve, fallacy, restricted, difficultie..."
63,63,blackassign0064,https://insights.blackcoffer.com/will-we-ever-...,Will we ever understand the nature of consciou...,understand nature consciousness ? introduction...,"[understand, nature, consciousness, ?, introdu...","[progress, trusted, pleasure, work, understand...","[controversial, mysterious, incomplete, worry,..."


In [69]:
def calculate_scores(tokens):
    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    return positive_score, negative_score

# Apply function to calculate scores
cleandf1['Positive_Score'], cleandf1['Negative_Score'] = zip(*cleandf1['Article_Text_Filtered_Tokenized'].apply(calculate_scores))
cleandf1.sample(10)

/tmp/ipykernel_25201/2230415470.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Positive_Score'], cleandf1['Negative_Score'] = zip(*cleandf1['Article_Text_Filtered_Tokenized'].apply(calculate_scores))
/tmp/ipykernel_25201/2230415470.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Positive_Score'], cleandf1['Negative_Score'] = zip(*cleandf1['Article_Text_Filtered_Tokenized'].apply(calculate_scores))


,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score
16,16,blackassign0017,https://insights.blackcoffer.com/rise-of-chatb...,Rise of Chatbots and its impact on customer su...,rise chatbots impact customer support year 204...,"[rise, chatbots, impact, customer, support, ye...","[support, intelligence, innovation, groundbrea...","[endanger, ignore, problems, discord, slack, c...",43,12
87,87,blackassign0088,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus impact on energy markets\n\nThe co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...","[significant, leading, led, work, energy-savin...","[emergency, disruption, downturn, hurts, virus...",16,44
91,91,blackassign0092,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the world...,estimating impact covid-19 world work covid19 ...,"[estimating, impact, covid-19, world, work, co...","[work, dominated, trump, trump, trump, success...","[stagnate, crisis, reject, rejection, worsen, ...",22,41
28,28,blackassign0029,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes\n...,ai healthcare improve patient outcomes introdu...,"[ai, healthcare, improve, patient, outcomes, i...","[improve, patient, robust, intelligence, prove...","[kills, virus, outbreak, invisible, virus, ove...",68,34
58,58,blackassign0059,https://insights.blackcoffer.com/can-robots-ta...,Can robots tackle late-life loneliness?\n\nIn ...,"robots tackle late-life loneliness ? era , bus...","[robots, tackle, late-life, loneliness, ?, era...","[satisfies, memorable, creative, work, masters...","[loneliness, loneliness, loneliness, lonelines...",23,37
82,82,blackassign0083,https://insights.blackcoffer.com/human-rights-...,Human Rights Outlook\n\nRead on to discover wh...,"human rights outlook read discover robot , ath...","[human, rights, outlook, read, discover, robot...","[foresight, recommend, confidence]",[issues],3,1
90,90,blackassign0091,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the world...,estimating impact covid-19 world work covid-19...,"[estimating, impact, covid-19, world, work, co...","[work, great, prodigious, great, fast, benefit...","[downturn, worst, recession, depression, crisi...",24,27
78,78,blackassign0079,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on Tourism & Aviat...,impact covid-19 pandemic tourism & aviation in...,"[impact, covid-19, pandemic, tourism, &, aviat...","[prefer, compliant, strong, supporting, tough,...","[problems, stress, conflicts, lost, incompatib...",21,32
8,8,blackassign0009,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect by the Year ...,rise cybercrime effect year 2040. technology c...,"[rise, cybercrime, effect, year, 2040., techno...","[benefits, genuine, sophisticated, advanced, a...","[concerns, challenging, scams, inevitably, sev...",38,50
7,7,blackassign0008,https://insights.blackcoffer.com/rise-of-inter...,Rise of Internet Demand and Its Impact on Comm...,rise internet demand impact communications alt...,"[rise, internet, demand, impact, communication...","[rapid, smart, affordable, significant, revolu...","[rival, break, latency, risks, threats, exploi...",30,9


In [70]:
# Calculate Polarity Score
cleandf1['Polarity_Score'] = (cleandf1['Positive_Score'] - cleandf1['Negative_Score']) / ((cleandf1['Positive_Score'] + cleandf1['Negative_Score']) + 0.000001)

# Display the dataframe with Polarity Score
cleandf1.sample(10)

/tmp/ipykernel_25201/1559554315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Polarity_Score'] = (cleandf1['Positive_Score'] - cleandf1['Negative_Score']) / ((cleandf1['Positive_Score'] + cleandf1['Negative_Score']) + 0.000001)


,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,Polarity_Score
47,47,blackassign0048,https://insights.blackcoffer.com/how-data-anal...,How Data Analytics can help your business resp...,data analytics business respond impact covid-1...,"[data, analytics, business, respond, impact, c...",[],"[failed, drastically, horrendous]",0,3,-1.000000
17,17,blackassign0018,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,rise e-health impact humans year 2030 introduc...,"[rise, e-health, impact, humans, year, 2030, i...","[promises, excitement, modern, patient, notabl...","[hype, epidemic, chronic, insecurity, difficul...",33,10,0.534884
0,0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"rising cities impact economy , environment , i...","[rising, cities, impact, economy, ,, environme...","[free, great, lead, good, good, good]",[harmful],6,1,0.714286
83,83,blackassign0084,https://insights.blackcoffer.com/how-voice-sea...,How Voice search makes your business a success...,voice search makes business successful busines...,"[voice, search, makes, business, successful, b...","[successful, easier, modern, passionate, popul...","[complex, friction, hacks, critical]",29,4,0.757576
92,92,blackassign0093,https://insights.blackcoffer.com/travel-and-to...,Travel and Tourism Outlook\n\nThe UN projects ...,travel tourism outlook projects 20-30 % declin...,"[travel, tourism, outlook, projects, 20-30, %,...","[good, foresight, recommend, confidence]","[decline, risk, issues]",4,3,0.142857
90,90,blackassign0091,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the world...,estimating impact covid-19 world work covid-19...,"[estimating, impact, covid-19, world, work, co...","[work, great, prodigious, great, fast, benefit...","[downturn, worst, recession, depression, crisi...",24,27,-0.058824
96,96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on office space an...,impact covid-19 pandemic office space co-worki...,"[impact, covid-19, pandemic, office, space, co...","[impeccable, great, improving, adequate, work,...","[sorrow, despise, torture, destruction, shunne...",22,35,-0.228070
1,1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"rising cities impact economy , environment , i...","[rising, cities, impact, economy, ,, environme...","[smart, improvements, advanced, revolutionary,...","[congestion, garbage, waste, disadvantages, di...",57,31,0.295455
39,39,blackassign0040,https://insights.blackcoffer.com/how-machines-...,"How Machines, AI, Automations, and Robo-human ...","machines , ai , automations , robo-human effec...","[machines, ,, ai, ,, automations, ,, robo-huma...","[effective, faster, rapid, intelligence, enhan...","[risk, risk, idle, risk, risk, risks, impossib...",37,22,0.254237
89,89,blackassign0090,https://insights.blackcoffer.com/lessons-from-...,Lessons from the past: Some key learnings rele...,lessons past : key learnings relevant coronavi...,"[lessons, past, :, key, learnings, relevant, c...","[ready, dumbfounded, simplest, clear, accurate...","[crisis, crisis, adversity, excuse, crisis, cr...",36,48,-0.142857


In [71]:
# Calculate Total Words after cleaning
cleandf1['Total_Words'] = cleandf1['Article_Text_Filtered'].apply(lambda x: len(x.split()))

# Calculate Subjectivity Score
cleandf1['Subjectivity_Score'] = (cleandf1['Positive_Score'] + cleandf1['Negative_Score']) / (cleandf1['Total_Words'] + 0.000001)
# Display the dataframe with Polarity Score
cleandf1.sample(10)

/tmp/ipykernel_25201/2194050375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Total_Words'] = cleandf1['Article_Text_Filtered'].apply(lambda x: len(x.split()))
/tmp/ipykernel_25201/2194050375.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Subjectivity_Score'] = (cleandf1['Positive_Score'] + cleandf1['Negative_Score']) / (cleandf1['Total_Words'] + 0.000001)


,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,Polarity_Score,Total_Words,Subjectivity_Score
49,49,blackassign0050,https://insights.blackcoffer.com/environmental...,Environmental impact of the COVID-19 pandemic ...,environmental impact covid-19 pandemic – lesso...,"[environmental, impact, covid-19, pandemic, –,...","[significant, important, positive, sharp, impo...","[infected, virus, imposing, strict, virus, adv...",24,69,-0.483871,829,0.112183
52,52,blackassign0053,https://insights.blackcoffer.com/how-python-be...,How Python became the first choice for Data Sc...,python choice data science . data science gain...,"[python, choice, data, science, ., data, scien...","[gaining, enjoying, recommendations, smoother,...","[hectic, worry, regression]",41,3,0.863636,395,0.111392
73,73,blackassign0074,https://insights.blackcoffer.com/gender-divers...,Gender diversity and Equality in the tech indu...,gender diversity equality tech industry gender...,"[gender, diversity, equality, tech, industry, ...","[important, rapid, important, sustainable, pos...","[limited, outcry, stranger, biased, queer, dis...",23,23,0.000000,691,0.066570
61,61,blackassign0062,https://insights.blackcoffer.com/are-we-any-cl...,Are we any closer to preventing a nuclear holo...,closer preventing nuclear holocaust ? thought ...,"[closer, preventing, nuclear, holocaust, ?, th...",[commitment],"[danger, threat, threat, critical, cold, risk,...",1,7,-0.750000,76,0.105263
34,34,blackassign0035,https://insights.blackcoffer.com/in-future-or-...,How humans and machines are evolving to work t...,humans machines evolving work ? future upcomin...,"[humans, machines, evolving, work, ?, future, ...","[work, work, work, work, work, flawlessly, eas...","[incapable, unimaginable, difficult, vice, dif...",27,12,0.384615,454,0.085903
44,44,blackassign0045,https://insights.blackcoffer.com/ai-healthcare...,How AI will impact the future of work?\n\nAI e...,ai impact future work ? ai experts ’ main driv...,"[ai, impact, future, work, ?, ai, experts, ’, ...","[work, pretty, improvements, renaissance, gold...","[doomsday, irresponsible, fiction, downside, s...",80,39,0.344538,804,0.148010
1,1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"rising cities impact economy , environment , i...","[rising, cities, impact, economy, ,, environme...","[smart, improvements, advanced, revolutionary,...","[congestion, garbage, waste, disadvantages, di...",57,31,0.295455,963,0.091381
68,68,blackassign0069,https://insights.blackcoffer.com/can-you-be-gr...,Can You Be Great Leader Without Technical Expe...,great leader technical expertise word “ leader...,"[great, leader, technical, expertise, word, “,...","[great, variety, passionate, inspiring, win, d...","[critical, critical, problem, problems, worse,...",31,6,0.675676,228,0.162281
42,42,blackassign0043,https://insights.blackcoffer.com/future-of-wor...,Future of Work: How AI Has Entered the Workpla...,future work : ai entered workplace artificial ...,"[future, work, :, ai, entered, workplace, arti...","[work, intelligence, clarity, benefits, intell...","[burning, issues, fear, virus, controversy, bi...",67,22,0.505618,952,0.093487
77,77,blackassign0078,https://insights.blackcoffer.com/impacts-of-co...,Impacts of COVID 19 on Vegetable Vendors\n\nTh...,impacts covid 19 vegetable vendors covid-19 pa...,"[impacts, covid, 19, vegetable, vendors, covid...","[healthy, fresh, fresh, dedicated, fresh, fres...","[disruptive, worried, catastrophic, interrupti...",25,18,0.162791,463,0.092873


In [75]:

from nltk.tokenize import sent_tokenize, word_tokenize

# Tokenize words and sentences
cleandf1['Sentences'] = cleandf1['Article_Text'].apply(sent_tokenize)
cleandf1['Words'] = cleandf1['Article_Text'].apply(word_tokenize)


# Calculate Average Sentence Length
cleandf1['Average_Sentence_Length'] = cleandf1['Words'].apply(lambda x: len(x) / len(cleandf1['Sentences']))

# Sample function to determine if a word is complex (you may need to customize this)
def is_complex(word):
    return len(word) > 6  # Example: consider words longer than 6 characters as complex

# Calculate Percentage of Complex Words
cleandf1['Complex_Words'] = cleandf1['Words'].apply(lambda x: sum(1 for word in x if is_complex(word)))
cleandf1['Percentage_Complex_Words'] = cleandf1['Complex_Words'] / len(cleandf1['Words'])

# Calculate Fog Index
cleandf1['Fog_Index'] = 0.4 * (cleandf1['Average_Sentence_Length'] + cleandf1['Percentage_Complex_Words'])

cleandf1.sample(10)

/tmp/ipykernel_25201/1218161883.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Sentences'] = cleandf1['Article_Text'].apply(sent_tokenize)
/tmp/ipykernel_25201/1218161883.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Words'] = cleandf1['Article_Text'].apply(word_tokenize)
/tmp/ipykernel_25201/1218161883.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,Polarity_Score,Total_Words,Subjectivity_Score,Sentences,Words,Average_Sentence_Length,Complex_Words,Percentage_Complex_Words,Fog_Index
64,64,blackassign0065,https://insights.blackcoffer.com/will-we-ever-...,Will we ever colonize outer space?\n\nNature h...,colonize outer space ? nature blessed humanity...,"[colonize, outer, space, ?, nature, blessed, h...","[survival, effective, boost, safely, led, prop...","[lies, dumping, damaging, petrified, subdued, ...",43,23,0.303030,883,0.074745,"[Will we ever colonize outer space?, Nature ha...","[Will, we, ever, colonize, outer, space, ?, Na...",19.397959,392,4.000000,9.359184
51,51,blackassign0052,https://insights.blackcoffer.com/difference-be...,What is the difference between Artificial Inte...,"difference artificial intelligence , machine l...","[difference, artificial, intelligence, ,, mach...","[intelligence, important, popular, fashionable...","[misconception, confusion, lies, fool, critici...",58,38,0.208333,1036,0.092664,[What is the difference between Artificial Int...,"[What, is, the, difference, between, Artificia...",21.826531,482,4.918367,10.697959
85,85,blackassign0086,https://insights.blackcoffer.com/how-to-increa...,How to increase social media engagement for ma...,increase social media engagement marketers ? s...,"[increase, social, media, engagement, marketer...","[likes, positive, popular, likes, good, benefi...","[hard, concerned, mystery, difficult, unknown,...",53,15,0.558824,1015,0.066995,[How to increase social media engagement for m...,"[How, to, increase, social, media, engagement,...",19.530612,450,4.591837,9.648980
38,38,blackassign0039,https://insights.blackcoffer.com/how-to-protec...,How to protect future data and its privacy?\n\...,"protect future data privacy ? internet , infor...","[protect, future, data, privacy, ?, internet, ...","[protect, improvements, modern, easy, won, pro...","[restricted, limits, hard, scramble, lost, sto...",46,64,-0.163636,1271,0.086546,"[How to protect future data and its privacy?, ...","[How, to, protect, future, data, and, its, pri...",22.775510,563,5.744898,11.408163
23,23,blackassign0024,https://insights.blackcoffer.com/rise-of-ott-p...,Rise of OTT platform and its impact on enterta...,rise ott platform impact entertainment industr...,"[rise, ott, platform, impact, entertainment, i...","[led, work, valuable, convenience, advantages,...","[senile, restricted, passive]",23,3,0.769231,363,0.071625,[Rise of OTT platform and its impact on entert...,"[Rise, of, OTT, platform, and, its, impact, on...",6.459184,179,1.826531,3.314286
65,65,blackassign0066,https://insights.blackcoffer.com/what-is-the-c...,What is the chance Homo sapiens will survive f...,chance homo sapiens survive 500 years ? ’ ve y...,"[chance, homo, sapiens, survive, 500, years, ?...","[sharp, masters, steady, optimistic, famed, in...","[insatiable, lost, disaster, irrepressible, fr...",34,26,0.133333,905,0.066298,[What is the chance Homo sapiens will survive ...,"[What, is, the, chance, Homo, sapiens, will, s...",17.091837,356,3.632653,8.289796
2,2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","internet demand ’ evolution , communication im...","[internet, demand, ’, evolution, ,, communicat...","[astounding, integral, promising, revolutioniz...","[limited, insatiable, lag, latency, lag, obsol...",38,24,0.225806,771,0.080415,"[Internet Demand’s Evolution, Communication Im...","[Internet, Demand, ’, s, Evolution, ,, Communi...",12.459184,458,4.673469,6.853061
70,70,blackassign0071,https://insights.blackcoffer.com/how-to-overco...,How to Overcome Your Fear of Making Mistakes\n...,overcome fear making mistakes reduce mistakes ...,"[overcome, fear, making, mistakes, reduce, mis...","[effective, smart, tough, visionary, visionary...","[fear, mista

In [76]:
# Calculate Average Number of Words Per Sentence
cleandf1['Average_Words_Per_Sentence'] = cleandf1.apply(lambda x: len(x['Words']) / len(x['Sentences']), axis=1)

# Calculate Complex Word Count
cleandf1['Complex_Word_Count'] = cleandf1['Words'].apply(lambda x: sum(1 for word in x if is_complex(word)))


/tmp/ipykernel_25201/2121032286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Average_Words_Per_Sentence'] = cleandf1.apply(lambda x: len(x['Words']) / len(x['Sentences']), axis=1)
/tmp/ipykernel_25201/2121032286.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Complex_Word_Count'] = cleandf1['Words'].apply(lambda x: sum(1 for word in x if is_complex(word)))


In [77]:
cleandf1.sample(10)

,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,...,Total_Words,Subjectivity_Score,Sentences,Words,Average_Sentence_Length,Complex_Words,Percentage_Complex_Words,Fog_Index,Average_Words_Per_Sentence,Complex_Word_Count
84,84,blackassign0085,https://insights.blackcoffer.com/how-the-covid...,How the COVID-19 crisis is redefining jobs and...,covid-19 crisis redefining jobs services ? pan...,"[covid-19, crisis, redefining, jobs, services,...","[long-lasting, work, work, work, work, importa...","[crisis, inevitable, slow, bad, concern, struc...",37,34,...,903,0.078627,[How the COVID-19 crisis is redefining jobs an...,"[How, the, COVID-19, crisis, is, redefining, j...",20.612245,441,4.500000,10.044898,16.833333,441
74,74,blackassign0075,https://insights.blackcoffer.com/how-to-overco...,How to overcome your fear of making mistakes?\...,overcome fear making mistakes ? covid-19 crisi...,"[overcome, fear, making, mistakes, ?, covid-19...","[effective, smart, tough, visionary, visionary...","[fear, mistakes, crisis, fallout, recession, u...",23,78,...,555,0.181982,[How to overcome your fear of making mistakes?...,"[How, to, overcome, your, fear, of, making, mi...",12.030612,239,2.438776,5.787755,16.150685,239
24,24,blackassign0025,https://insights.blackcoffer.com/rise-of-elect...,Rise of Electric Vehicles and its Impact on Li...,rise electric vehicles impact livelihood 2040 ...,"[rise, electric, vehicles, impact, livelihood,...","[good, exceeds, promises, electrify, ambitious...","[bad, unhealthy, crude, hasty, worry, issue, a...",19,11,...,615,0.048780,[Rise of Electric Vehicles and its Impact on L...,"[Rise, of, Electric, Vehicles, and, its, Impac...",11.846939,248,2.530612,5.751020,46.440000,248
90,90,blackassign0091,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the world...,estimating impact covid-19 world work covid-19...,"[estimating, impact, covid-19, world, work, co...","[work, great, prodigious, great, fast, benefit...","[downturn, worst, recession, depression, crisi...",24,27,...,560,0.091071,[Estimating the impact of COVID-19 on the worl...,"[Estimating, the, impact, of, COVID-19, on, th...",11.959184,298,3.040816,6.000000,31.675676,298
7,7,blackassign0008,https://insights.blackcoffer.com/rise-of-inter...,Rise of Internet Demand and Its Impact on Comm...,rise internet demand impact communications alt...,"[rise, internet, demand, impact, communication...","[rapid, smart, affordable, significant, revolu...","[rival, break, latency, risks, threats, exploi...",30,9,...,589,0.066214,[Rise of Internet Demand and Its Impact on Com...,"[Rise, of, Internet, Demand, and, Its, Impact,...",9.061224,347,3.540816,5.040816,18.893617,347
56,56,blackassign0057,https://insights.blackcoffer.com/telemedicine-...,What patients like and dislike about telemedic...,patients dislike telemedicine ? today ’ world ...,"[patients, dislike, telemedicine, ?, today, ’,...","[remarkable, great, pretty, satisfied, pretty,...","[dislike, hectic, concerned, doubt, distrust, ...",15,9,...,200,0.120000,[What patients like and dislike about telemedi...,"[What, patients, like, and, dislike, about, te...",4.428571,107,1.091837,2.208163,15.500000,107
95,95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,Due to the COVID-19 the repercussion of the en...,due covid-19 repercussion environment epidemic...,"[due, covid-19, repercussion, environment, epi...","[lead, significant, important, leading, advanc...","[epidemic, outbreak, setback, toll, fall, viru...",28,57,...,665,0.127820,[Due to the COVID-19 the repercussion of the e...,"[Due, to, the, COVID-19, the, repercussion, of...",12.387755,365,3.724490,6.444898,24.280000,365
83,83,blackassign0084,https://insights.blackcoffer.com/how-voice-sea...,How Voice search makes your business a success...,voice search makes business successful busines...,"[voice, search, mak

In [79]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load English stopwords and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

def clean_and_count_words(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Remove stopwords and punctuation
    words = [word for word in words if word not in stop_words and word not in punctuation]
    # Return the count of cleaned words
    return len(words)


# Calculate Word Count after cleaning
cleandf1['Word_Count'] = cleandf1['Article_Text'].apply(clean_and_count_words)

cleandf1.sample(10)



[nltk_data] Downloading package punkt to /home/danjin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/danjin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_25201/3885663631.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Word_Count'] = cleandf1['Article_Text'].apply(clean_and_count_words)


,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,...,Subjectivity_Score,Sentences,Words,Average_Sentence_Length,Complex_Words,Percentage_Complex_Words,Fog_Index,Average_Words_Per_Sentence,Complex_Word_Count,Word_Count
65,65,blackassign0066,https://insights.blackcoffer.com/what-is-the-c...,What is the chance Homo sapiens will survive f...,chance homo sapiens survive 500 years ? ’ ve y...,"[chance, homo, sapiens, survive, 500, years, ?...","[sharp, masters, steady, optimistic, famed, in...","[insatiable, lost, disaster, irrepressible, fr...",34,26,...,0.066298,[What is the chance Homo sapiens will survive ...,"[What, is, the, chance, Homo, sapiens, will, s...",17.091837,356,3.632653,8.289796,20.426829,356,835
66,66,blackassign0067,https://insights.blackcoffer.com/why-does-your...,Why does your business need a chatbot?\n\nMr. ...,business chatbot ? mr. sakamoto bonsai artist ...,"[business, chatbot, ?, mr., sakamoto, bonsai, ...","[loves, loved, loved, happy, satisfied, awesom...","[remorse, sadness, problems, stuck, limit, exp...",24,7,...,0.072600,"[Why does your business need a chatbot?, Mr. S...","[Why, does, your, business, need, a, chatbot, ...",9.918367,200,2.040816,4.783673,15.677419,200,391
24,24,blackassign0025,https://insights.blackcoffer.com/rise-of-elect...,Rise of Electric Vehicles and its Impact on Li...,rise electric vehicles impact livelihood 2040 ...,"[rise, electric, vehicles, impact, livelihood,...","[good, exceeds, promises, electrify, ambitious...","[bad, unhealthy, crude, hasty, worry, issue, a...",19,11,...,0.048780,[Rise of Electric Vehicles and its Impact on L...,"[Rise, of, Electric, Vehicles, and, its, Impac...",11.846939,248,2.530612,5.751020,46.440000,248,617
19,19,blackassign0020,https://insights.blackcoffer.com/how-advertise...,How advertisement increase your market value?\...,"advertisement increase market ? research , adv...","[advertisement, increase, market, ?, research,...","[famous, success, successfully, faith, success...",[],8,0,...,0.030418,[How advertisement increase your market value?...,"[How, advertisement, increase, your, market, v...",6.020408,129,1.316327,2.934694,22.692308,129,275
87,87,blackassign0088,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus impact on energy markets\n\nThe co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...","[significant, leading, led, work, energy-savin...","[emergency, disruption, downturn, hurts, virus...",16,44,...,0.054446,[Coronavirus impact on energy markets\n\nThe c...,"[Coronavirus, impact, on, energy, markets, The...",19.346939,544,5.551020,9.959184,31.600000,544,1034
1,1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"rising cities impact economy , environment , i...","[rising, cities, impact, economy, ,, environme...","[smart, improvements, advanced, revolutionary,...","[congestion, garbage, waste, disadvantages, di...",57,31,...,0.091381,[Rising IT Cities and Their Impact on the Econ...,"[Rising, IT, Cities, and, Their, Impact, on, t...",16.622449,460,4.693878,8.526531,21.155844,460,864
79,79,blackassign0080,https://insights.blackcoffer.com/impact-of-cov...,Impact of COVID-19 pandemic on sports events a...,impact covid-19 pandemic sports events world ....,"[impact, covid-19, pandemic, sports, events, w...","[significant, super, gold, gold, fans, fans, d...","[disruption, outbreak, tentatively, limited, d...",36,11,...,0.017803,[Impact of COVID-19 pandemic on sports events ...,"[Impact, of, COVID-19, pandemic, on, sports, e...",40.561224,835,8.520408,19.632653,20.384615,835,2273
91,91,blackassign0092,https://insights.blackcoffer.com/estimating-th...,Estimating the impact of COVID-19 on the world...,estimating impact covid-19 world work covid19 ...,"[estimating, impact, covid-19, world, work, co...","[work, dom

In [82]:
import re

# Function to count syllables in a word
def syllable_count(word):
    # Remove common suffixes that do not contribute to syllable count
    word = re.sub(r'e$', '', word)
    word = re.sub(r'ed$', '', word)
    word = re.sub(r'es$', '', word)
    
    # Count vowels (not including 'y' if it is the only vowel in the syllable)
    vowels = 'aeiou'
    count = 0
    last_char_was_vowel = False
    for char in word:
        if char in vowels and not last_char_was_vowel:
            count += 1
            last_char_was_vowel = True
        elif char not in vowels:
            last_char_was_vowel = False
    
    # Handle special case when word ends with 'le' as a syllable
    if word.endswith('le') and len(word) > 2 and word[-3] not in vowels:
        count += 1
    
    return count

# Function to count personal pronouns
def count_personal_pronouns(text):
    pronouns = ['i', 'we', 'my', 'ours', 'us']
    # Use regex to find the counts of the pronouns
    return sum(1 for pronoun in pronouns if re.search(r'\b' + pronoun + r'\b', text.lower()))


In [83]:
    # Calculate Syllable Count Per Word
cleandf1['Syllable_Count_Per_Word'] = cleandf1['Article_Text'].apply(lambda x: [syllable_count(word) for word in x.split()])

# Calculate Personal Pronouns
cleandf1['Personal_Pronouns_Count'] = cleandf1['Article_Text'].apply(count_personal_pronouns)

# Calculate Average Word Length
cleandf1['Average_Word_Length'] = cleandf1['Article_Text'].apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()))

cleandf1.sample(10)

/tmp/ipykernel_25201/3939088303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Syllable_Count_Per_Word'] = cleandf1['Article_Text'].apply(lambda x: [syllable_count(word) for word in x.split()])
/tmp/ipykernel_25201/3939088303.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandf1['Personal_Pronouns_Count'] = cleandf1['Article_Text'].apply(count_personal_pronouns)
/tmp/ipykernel_25201/3939088303.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,Unnamed: 0,URL_ID,URL,Article_Text,Article_Text_Filtered,Article_Text_Filtered_Tokenized,Positive_Words,Negative_Words,Positive_Score,Negative_Score,...,Average_Sentence_Length,Complex_Words,Percentage_Complex_Words,Fog_Index,Average_Words_Per_Sentence,Complex_Word_Count,Word_Count,Syllable_Count_Per_Word,Personal_Pronouns_Count,Average_Word_Length
2,2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand’s Evolution, Communication Imp...","internet demand ’ evolution , communication im...","[internet, demand, ’, evolution, ,, communicat...","[astounding, integral, promising, revolutioniz...","[limited, insatiable, lag, latency, lag, obsol...",38,24,...,12.459184,458,4.673469,6.853061,21.803571,458,676,"[2, 2, 3, 5, 1, 1, 0, 3, 2, 3, 0, 0, 1, 1, 1, ...",2,6.280751
74,74,blackassign0075,https://insights.blackcoffer.com/how-to-overco...,How to overcome your fear of making mistakes?\...,overcome fear making mistakes ? covid-19 crisi...,"[overcome, fear, making, mistakes, ?, covid-19...","[effective, smart, tough, visionary, visionary...","[fear, mistakes, crisis, fallout, recession, u...",23,78,...,12.030612,239,2.438776,5.787755,16.150685,239,536,"[1, 1, 3, 1, 1, 1, 2, 3, 0, 2, 2, 1, 1, 2, 3, ...",4,4.935707
76,76,blackassign0077,https://insights.blackcoffer.com/impacts-of-co...,Impacts of COVID 19 on Food products\n\nSome v...,impacts covid 19 food products vendors ( fruit...,"[impacts, covid, 19, food, products, vendors, ...","[eased, relief, sufficient, benefit, support, ...","[harassment, risk, harsh, unable, loss, crisis]",8,6,...,3.346939,77,0.785714,1.653061,29.818182,77,159,"[1, 1, 0, 0, 1, 1, 2, 1, 2, 1, 1, 3, 2, 2, 3, ...",0,4.979730
15,15,blackassign0016,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,rise telemedicine impact livelihood 2040 “ gai...,"[rise, telemedicine, impact, livelihood, 2040,...","[gains, accessible, comfort, capable, prompt, ...","[fall, inequities, emergency, obscure, complex...",34,27,...,13.918367,437,4.459184,7.351020,23.118644,437,764,"[1, 1, 5, 1, 1, 1, 1, 4, 0, 0, 1, 1, 1, 2, 5, ...",2,5.621027
55,55,blackassign0056,https://insights.blackcoffer.com/impact-of-ai-...,Impact of AI in health and medicine\n\nAI allo...,impact ai health medicine ai training naturali...,"[impact, ai, health, medicine, ai, training, n...","[instantly, profound, support, support, patien...","[unimaginable, symptoms, illness, symptoms, ur...",13,8,...,5.489796,152,1.551020,2.816327,28.315789,152,287,"[1, 1, 0, 1, 1, 1, 3, 0, 2, 1, 1, 2, 1, 1, 1, ...",1,5.385593
34,34,blackassign0035,https://insights.blackcoffer.com/in-future-or-...,How humans and machines are evolving to work t...,humans machines evolving work ? future upcomin...,"[humans, machines, evolving, work, ?, future, ...","[work, work, work, work, work, flawlessly, eas...","[incapable, unimaginable, difficult, vice, dif...",27,12,...,8.571429,204,2.081633,4.261224,18.666667,204,411,"[1, 2, 1, 2, 1, 3, 1, 1, 3, 0, 2, 1, 1, 3, 1, ...",2,5.204082
44,44,blackassign0045,https://insights.blackcoffer.com/ai-healthcare...,How AI will impact the future of work?\n\nAI e...,ai impact future work ? ai experts ’ main driv...,"[ai, impact, future, work, ?, ai, experts, ’, ...","[work, pretty, improvements, renaissance, gold...","[doomsday, irresponsible, fiction, downside, s...",80,39,...,19.285714,421,4.295918,9.432653,210.000000,421,949,"[1, 0, 1, 2, 0, 2, 1, 1, 0, 2, 2, 1, 1, 1, 0, ...",3,4.806648
21,21,blackassign0022,https://insights.blackcoffer.com/how-advertise...,How advertisement/marketing affects business.\...,advertisement/marketing affects business . liv...,"[advertisement/marketing, affects, business, ....","[clear, respect, ample, competitive, dynamic, ...","[failure, wrong, harmful, wrong, unnecessary, ...",12,10,...,4.683673,113,1.153061,2.334694,17.000000,113,211,"[1, 8, 2, 3, 0, 1, 2, 1, 0, 1, 3, 1, 1, 1, 0, ...",2,5.111380
28,28,blackassign0029,https://insights.blackcoff

In [84]:
cleandf1.to_csv('final.csv')